In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import subprocess

# Define class(es)

In [2]:
class Command():
    '''
    Creates a command and an empty command list for UNIX command line programs/applications. Primary use and
    use-cases are intended for the subprocess module and its associated classes (i.e. call/run).
    Attributes:
        command: Command to be performed on the command line
    '''

    def __init__(self):
        '''
        Init doc-string for Command class.
        '''
        pass

    def init_cmd(self, command):
        '''
        Init command function for initializing commands to be used on UNIX command line.
        
        Arguments:
            command (string): Command to be used. Note: command used must be in system path
        Returns:
            cmd_list (list): Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        return self.cmd_list

In [3]:
def run(cmd_list,stdout="",stderr=""):
    '''
    Uses python's built-in subprocess class to run a command from an input command list.
    The standard output and error can optionally be written to file.
    
    Arguments:
        cmd_list(list): Input command list to be run from the UNIX command line.
        stdout(file): Output file to write standard output to.
        stderr(file): Output file to write standard error to.
    Returns:
        stdout(file): Output file that contains the standard output.
        stderr(file): Output file that contains the standard error.
    '''
    if stdout and stderr:
        with open(stdout,"w") as file:
            with open(stderr,"w") as file_err:
                subprocess.call(cmd_list,stdout=file,stderr=file_err)
                file.close(); file_err.close()
    elif stdout:
        with open(stdout,"w") as file:
            subprocess.call(cmd_list,stdout=file)
            file.close()
        stderr = None
    else:
        subprocess.call(cmd_list)
        stdout = None
        stderr = None

    return stdout,stderr

# Work Space
-----
Work space for testing functions

In [4]:
csv_file = "infant-neo-aal.csv"

In [21]:
def read_atlas_file(atlas_info):
    '''
    working doc-string
    '''
    
    atlas_info = os.path.abspath(atlas_info)
    
    atlas_info = os.path.abspath(atlas_info)
    df = pd.read_csv(atlas_info,header=None); df.columns = ['key', 'id']
    atlas_dict = df.set_index('key').to_dict(orient='dict')['id']
    
    return atlas_dict

In [6]:
nii_atlas = "infant-neo-aal-2mm.nii"
# nii_atlas = "infant-neo-aal-2mm_int.nii"

In [7]:
# img = nib.load(nii_atlas)

In [8]:
# print(img.header)

In [9]:
# img.get_data

In [10]:
# data = img.get_fdata()
# data.astype(int)

In [11]:
# type(data)

In [12]:
# len(data)

In [13]:
# data.shape

In [14]:
# len(np.unique(data.astype(int)))

In [15]:
# len(np.unique(data.astype(int)))

In [17]:
def remove_ext(file):
    '''
    working doc-string
    '''
    
    if '.nii.gz' in file:
        name = file[:-7]
    elif '.nii' in file:
        name = file[:-4]
    elif '.txt' in file or '.tsv' in file or '.csv' in file:
        name = file
    else:
        name = file
    
    return name

In [18]:
def convert_img_dtype(img,data_type="int"):
    '''
    working doc-string
    '''
    
    # remove_ext, use fslmaths to convert image from float to int
    img = os.path.abspath(img)
    out_dir = os.path.dirname(img)
    
    img = remove_ext(img)
    name = os.path.basename(img)
    
    out_file = os.path.join(out_dir,name + ".int" + ".nii.gz")
    
    img_dtype = Command().init_cmd("fslmaths")
    img_dtype.append("-dt")
    img_dtype.append(data_type)
    img_dtype.append(img)
    img_dtype.append(out_file)
    img_dtype.append("-odt")
    img_dtype.append(data_type)
    
    run(img_dtype)
    
    return out_file

In [19]:
def load_atlas_data(nii_atlas,atlas_info,data_type="int"):
    '''
    working doc-string
    '''
    
    # Load atlas key/ID information
    atlas_dict = read_atlas_file(atlas_info)
    
    # Convert input image from float to int
    int_data = convert_img_dtype(nii_atlas,data_type)
    
    # Load/export data as numpy array
    img = nib.load(int_data)
    atlas_data = img.get_fdata()
    
    # Clean-up
    os.remove(int_data)
    
    return atlas_data,atlas_dict

In [20]:
[atlas_data,atlas_dict] = load_atlas_data(nii_atlas,csv_file)

In [22]:
def make_cluster_vol(nii_file,thresh=0.95,dist=0):
    '''
    working doc-string
    '''
    
    # Construct file paths
    nii_file = os.path.abspath(nii_file)
    out_dir = os.path.dirname(nii_file)
    
    nii_file = remove_ext(nii_file)
    name = os.path.basename(nii_file)
    out_prefix = os.path.join(out_dir,name + ".cluster")
    
    out_file = out_prefix + ".nii.gz"
    out_stat = out_prefix + ".txt"
    
    # Make cluster(s) from input data
    clust_cmd = Command.init_cmd("cluster")
    clust_cmd.append(f"--input={nii_file}")
    clust_cmd.append(f"--thresh={thresh}")
    clust_cmd.append(f"--peakdist={dist}")
    clust_cmd.append(f"--othresh={out_file}")
    
    run(clust_cmd,out_stat)
    
    return out_file,out_stat